# _Randomly Connected Neural Networks for Self-Supervised Monocular Depth Estimation_ Demo Notebook

## Imports

In [1]:
! pip install --upgrade pip
! pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.1.2
    Uninstalling pip-22.1.2:
      Successfully uninstalled pip-22.1.2


In [2]:
from matplotlib import pyplot as plt

from PIL import Image

import torch
from torch.utils.data import DataLoader
from torchvision import transforms

import yaml

from loaders import CityScapesDataset

from model import RandomlyConnectedModel

import train
from train.loss import MonodepthLoss

## Setup

### CUDA

In [3]:
device = torch.device("cuda") \
    if torch.cuda.is_available() \
        else torch.device("cpu")

print(device)

cuda


### Config file

In [4]:
with open("config.yml") as f:
    model_config = yaml.load(f, Loader=yaml.Loader)

encoder_config = model_config["encoder"]
decoder_config = model_config["decoder"]

### Hyperparameters

In [23]:
# Dataset parameters
batch_size = 8
validation_samples = 1000
numberof_workers = 8

# Training parameters
numberof_epochs = 30
learning_rate = 1e-4

## Dataset

### Transforms

In [24]:
train_transform = transforms.Compose([
    train.transforms.ResizeImage((256, 512)),
    train.transforms.RandomFlip(0.5),
    train.transforms.ToTensor(),
    train.transforms.RandomAugment(0.5, gamma=(0.8, 1.2),
                                   brightness=(0.5, 2.0),
                                   colour=(0.8, 1.2))
])

val_transform = transforms.Compose([
    train.transforms.ResizeImage((256, 512)),
    train.transforms.ToTensor()
])

### CityScapes

In [25]:
train_dataset = CityScapesDataset("../datasets/cityscapes/", "train",
                                  train_transform)
                                  
val_dataset = CityScapesDataset("../datasets/cityscapes/", "val",
                                val_transform, validation_samples)

print(f"Dataset size:"
      f"\n\tTrain: {len(train_dataset):,} images."
      f"\n\tTest: {len(val_dataset):,} images.")

Dataset size:
	Train: 2,975 images.
	Test: 500 images.


### Loaders

In [26]:
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          shuffle=True, num_workers=numberof_workers)

val_loader = DataLoader(val_dataset, batch_size=batch_size,
                        shuffle=True, num_workers=numberof_workers)

## Model

### Initialisation

In [30]:
# Temporary solution until config code is set up
model = RandomlyConnectedModel(load_graph="graphs/nodes_5_seed_42").to(device)
loss_function = MonodepthLoss().to(device)

numberof_parameters = sum(p.numel() for p in model.parameters())
print(f"Model has {numberof_parameters:,} learnable parameters.")
print(f"Using CUDA? {next(model.parameters()).is_cuda}")

Model has 22,818,044 learnable parameters.
Using CUDA? True


### Train

In [28]:
train.train_model(model, train_loader, loss_function, numberof_epochs,
                  learning_rate, val_loader=val_loader, evaluate_every=10, 
                  save_path="trained/", save_every=2,
                  save_comparison="results/",
                  device=device)

Epoch #1:   2%|▏         | 9/372 [00:56<38:17,  6.33s/batch, loss=1.8e+5]   


KeyboardInterrupt: 

### Evaluate

In [ ]:
model.eval()

loss_function = MonodepthLoss().to(device)

train.evaluate_model(model, val_loader, loss_function,
                     save_comparison="results/",
                     device=device)

### Results

In [ ]:
image = Image.open("results/comparison.png")
plt.imshow(image)